# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [1]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Array{Float64,1}:
 1.8569052513607969
 1.4435006853770964
 1.5570405899570547

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [2]:
Alu = lu(A)

LU{Float64,Array{Float64,2}}
L factor:
3×3 Array{Float64,2}:
 1.0       0.0       0.0
 0.855966  1.0       0.0
 0.54739   0.402517  1.0
U factor:
3×3 Array{Float64,2}:
 0.888313  0.00191593   0.966677
 0.0       0.376817    -0.522763
 0.0       0.0          0.599336

In [3]:
typeof(Alu)

LU{Float64,Array{Float64,2}}

The different parts of the factorization can be extracted by accessing their special properties

In [4]:
Alu.P

3×3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

In [5]:
Alu.L

3×3 Array{Float64,2}:
 1.0       0.0       0.0
 0.855966  1.0       0.0
 0.54739   0.402517  1.0

In [6]:
Alu.U

3×3 Array{Float64,2}:
 0.888313  0.00191593   0.966677
 0.0       0.376817    -0.522763
 0.0       0.0          0.599336

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [7]:
A\b

3-element Array{Float64,1}:
 1.0               
 1.0000000000000002
 1.0               

In [8]:
Alu\b

3-element Array{Float64,1}:
 1.0               
 1.0000000000000002
 1.0               

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [10]:
det(A) == det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [11]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.701463   0.688295  -0.184931
 -0.600428  -0.71051   -0.366962
 -0.383974  -0.146373   0.911668
R factor:
3×3 Array{Float64,2}:
 -1.26637  -0.287222  -1.21354 
  0.0      -0.289933   0.314502
  0.0       0.0        0.546396

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [12]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.701463   0.688295  -0.184931
 -0.600428  -0.71051   -0.366962
 -0.383974  -0.146373   0.911668

In [13]:
Aqr.R

3×3 Array{Float64,2}:
 -1.26637  -0.287222  -1.21354 
  0.0      -0.289933   0.314502
  0.0       0.0        0.546396

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [14]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
3-element Array{Float64,1}:
 0.19566171494095277
 0.6458510681219253 
 3.528152532854935  
eigenvectors:
3×3 Array{Float64,2}:
  0.688659  -0.243637  -0.682928
 -0.567115  -0.767863  -0.297937
 -0.451807   0.592476  -0.666966

The values and the vectors can be extracted from the Eigen type by special indexing

In [15]:
AsymEig.values

3-element Array{Float64,1}:
 0.19566171494095277
 0.6458510681219253 
 3.528152532854935  

In [16]:
AsymEig.vectors

3×3 Array{Float64,2}:
  0.688659  -0.243637  -0.682928
 -0.567115  -0.767863  -0.297937
 -0.451807   0.592476  -0.666966

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [18]:
inv(AsymEig)*Asym

3×3 Array{Float64,2}:
  1.0           1.33227e-15  -4.44089e-16
  9.99201e-16   1.0          -2.22045e-16
 -8.88178e-16  -1.66533e-15   1.0        

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [19]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [20]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [21]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

0.28121166368078815

In [22]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [23]:
?Symmetric

search: Symmetric issymmetric



```
Symmetric(A, uplo=:U)
```

Construct a `Symmetric` view of the upper (if `uplo = :U`) or lower (if `uplo = :L`) triangle of the matrix `A`.

# Examples

```jldoctest
julia> A = [1 0 2 0 3; 0 4 0 5 0; 6 0 7 0 8; 0 9 0 1 0; 2 0 3 0 4]
5×5 Array{Int64,2}:
 1  0  2  0  3
 0  4  0  5  0
 6  0  7  0  8
 0  9  0  1  0
 2  0  3  0  4

julia> Supper = Symmetric(A)
5×5 Symmetric{Int64,Array{Int64,2}}:
 1  0  2  0  3
 0  4  0  5  0
 2  0  7  0  8
 0  5  0  1  0
 3  0  8  0  4

julia> Slower = Symmetric(A, :L)
5×5 Symmetric{Int64,Array{Int64,2}}:
 1  0  6  0  2
 0  4  0  9  0
 6  0  7  0  3
 0  9  0  1  0
 2  0  3  0  4
```

Note that `Supper` will not be equal to `Slower` unless `A` is itself symmetric (e.g. if `A == transpose(A)`).


In [25]:
Asym_explicit = Symmetric(Asym_noisy)

1000×1000 Symmetric{Float64,Array{Float64,2}}:
  3.67908     0.281212    0.385946   …  -0.167902   -0.637034   -2.11801  
  0.281212    0.820701    0.801522       1.78756    -1.98148    -0.0943299
  0.385946    0.801522   -4.09416       -0.354785   -1.54445     0.137818 
  1.25749    -1.88387    -0.165771       1.95611     0.485965   -0.491188 
  1.1032     -2.86652     2.12382        0.22963    -0.58062    -0.387873 
 -0.226168    0.247323    0.362259   …  -0.447125   -0.957669    0.158319 
  0.201396    2.21439    -1.43376       -0.808806   -0.72335    -1.26078  
 -1.58761     0.561689   -0.0653849     -1.78711     0.383983    0.793298 
 -1.91315    -0.0734975   0.484143       1.47337    -0.625338   -1.08886  
 -0.974388   -3.28833     0.39747        1.68167    -2.32289     1.33097  
 -0.174364    1.11363     0.850635   …  -0.0297972   1.36081    -0.0907651
  2.21165     0.350801   -0.726477      -2.9432     -0.32611     1.54643  
 -1.967       0.0101535   1.49311        1.29642     

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [32]:
@time eigvals(Asym);

  0.262729 seconds (16 allocations: 7.989 MiB, 1.58% gc time)


In [30]:
@time eigvals(Asym_noisy);

  1.419182 seconds (18 allocations: 7.921 MiB)


In [33]:
@time eigvals(Asym_explicit);

  0.160697 seconds (16 allocations: 7.989 MiB)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [35]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.744087 seconds (49 allocations: 183.107 MiB, 8.98% gc time)


6.465312521764517

In [37]:
@time eigmax(A)

  0.733038 seconds (49 allocations: 183.107 MiB, 11.30% gc time)


6.465312521764517

In [38]:
A

1000000×1000000 SymTridiagonal{Float64,Array{Float64,1}}:
 0.278628   0.997564   ⋅          ⋅        …    ⋅          ⋅         ⋅      
 0.997564  -1.39328   0.706969    ⋅             ⋅          ⋅         ⋅      
  ⋅         0.706969  0.103376   1.03975        ⋅          ⋅         ⋅      
  ⋅          ⋅        1.03975    0.195276       ⋅          ⋅         ⋅      
  ⋅          ⋅         ⋅        -1.30052        ⋅          ⋅         ⋅      
  ⋅          ⋅         ⋅          ⋅        …    ⋅          ⋅         ⋅      
  ⋅          ⋅         ⋅          ⋅             ⋅          ⋅         ⋅      
  ⋅          ⋅         ⋅          ⋅             ⋅          ⋅         ⋅      
  ⋅          ⋅         ⋅          ⋅             ⋅          ⋅         ⋅      
  ⋅          ⋅         ⋅          ⋅             ⋅          ⋅         ⋅      
  ⋅          ⋅         ⋅          ⋅        …    ⋅          ⋅         ⋅      
  ⋅          ⋅         ⋅          ⋅             ⋅          ⋅         ⋅      
  ⋅          ⋅    

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [ ]:
1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [43]:
Arational = Matrix{Rational{BigInt}}(rand(1:10000, 3, 3))/10000

3×3 Array{Rational{BigInt},2}:
  197//5000   8903//10000   297//10000
 4317//10000   358//625    4719//10000
 4571//5000    267//400    1941//10000

In [44]:
x = fill(1, 3)
b = Arational*x

3-element Array{Rational{BigInt},1}:
 4797//5000
 3691//2500
 8879//5000

In [45]:
Arational\b

3-element Array{Rational{BigInt},1}:
 1//1
 1//1
 1//1

In [46]:
lu(Arational)

LU{Rational{BigInt},Array{Rational{BigInt},2}}
L factor:
3×3 Array{Rational{BigInt},2}:
    1//1           0//1         0//1
 4317//394         1//1         0//1
 4571//197  78761276//36177419  1//1
U factor:
3×3 Array{Rational{BigInt},2}:
 197//5000       8903//10000             297//10000      
   0//1     -36177419//3940000        577137//3940000    
   0//1             0//1        -73614979767//90443547500

In [51]:
H = [big(1)//(i+j-1) for i=1:20,j=1:20]

20×20 Array{Rational{BigInt},2}:
 1//1   1//2   1//3   1//4   1//5   …  1//16  1//17  1//18  1//19  1//20
 1//2   1//3   1//4   1//5   1//6      1//17  1//18  1//19  1//20  1//21
 1//3   1//4   1//5   1//6   1//7      1//18  1//19  1//20  1//21  1//22
 1//4   1//5   1//6   1//7   1//8      1//19  1//20  1//21  1//22  1//23
 1//5   1//6   1//7   1//8   1//9      1//20  1//21  1//22  1//23  1//24
 1//6   1//7   1//8   1//9   1//10  …  1//21  1//22  1//23  1//24  1//25
 1//7   1//8   1//9   1//10  1//11     1//22  1//23  1//24  1//25  1//26
 1//8   1//9   1//10  1//11  1//12     1//23  1//24  1//25  1//26  1//27
 1//9   1//10  1//11  1//12  1//13     1//24  1//25  1//26  1//27  1//28
 1//10  1//11  1//12  1//13  1//14     1//25  1//26  1//27  1//28  1//29
 1//11  1//12  1//13  1//14  1//15  …  1//26  1//27  1//28  1//29  1//30
 1//12  1//13  1//14  1//15  1//16     1//27  1//28  1//29  1//30  1//31
 1//13  1//14  1//15  1//16  1//17     1//28  1//29  1//30  1//31  1//32
 1//14  1//15  1//

In [53]:
inv(H)

20×20 Array{Rational{BigInt},2}:
              400//1  …               -1378465288200//1
           -79800//1                 523816809516000//1
          5266800//1              -49500688499262000//1
       -171609900//1             2057028610969332000//1
       3294910080//1           -47311658052294636000//1
     -41186376000//1  …        681287875953042758400//1
     356948592000//1         -6623632127321249040000//1
   -2237302782000//1         45689544061930248480000//1
   10440746316000//1       -231303316813521882930000//1
  -37006645275600//1        879523723192157283240000//1
  100927214388000//1  …   -2550618797257256121396000//1
 -213323430411000//1       5691463431896356634520000//1
  350069219136000//1      -9801964799377058648340000//1
 -444318624288000//1      12991953343553024480640000//1
  431623806451200//1     -13124524296038259424320000//1
 -314725692204000//1  …    9916307245895573787264000//1
  166619484108000//1      -5422980525099141914910000//1
  -604404010980

In [54]:
using Random

In [55]:
?Random.seed!

```
seed!([rng=GLOBAL_RNG], seed) -> rng
seed!([rng=GLOBAL_RNG]) -> rng
```

Reseed the random number generator: `rng` will give a reproducible sequence of numbers if and only if a `seed` is provided. Some RNGs don't accept a seed, like `RandomDevice`. After the call to `seed!`, `rng` is equivalent to a newly created object initialized with the same seed.

# Examples

```julia-repl
julia> Random.seed!(1234);

julia> x1 = rand(2)
2-element Array{Float64,1}:
 0.590845
 0.766797

julia> Random.seed!(1234);

julia> x2 = rand(2)
2-element Array{Float64,1}:
 0.590845
 0.766797

julia> x1 == x2
true

julia> rng = MersenneTwister(1234); rand(rng, 2) == x1
true

julia> MersenneTwister(1) == Random.seed!(rng, 1)
true

julia> rand(Random.seed!(rng), Bool) # not reproducible
true

julia> rand(Random.seed!(rng), Bool)
false

julia> rand(MersenneTwister(), Bool) # not reproducible either
true
```


### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
and assign it a variable `A_eigv`

In [ ]:
using LinearAlgebra

In [56]:
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]

5×5 Array{Int64,2}:
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36

In [57]:
A_eigv = eigen(A).values

5-element Array{Float64,1}:
 -128.49322764802145 
  -55.887784553057   
   42.752167279318854
   87.16111477514494 
  542.4677301466137  

In [ ]:
@assert A_eigv ==  [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [58]:
A_diag = Diagonal(A_eigv)

5×5 Diagonal{Float64,Array{Float64,1}}:
 -128.493     ⋅        ⋅        ⋅         ⋅   
     ⋅     -55.8878    ⋅        ⋅         ⋅   
     ⋅        ⋅      42.7522    ⋅         ⋅   
     ⋅        ⋅        ⋅      87.1611     ⋅   
     ⋅        ⋅        ⋅        ⋅      542.468

In [ ]:
@assert A_diag ==  [-128.493    0.0      0.0      0.0       0.0;
    0.0    -55.8878   0.0      0.0       0.0;
    0.0      0.0     42.7522   0.0       0.0;
    0.0      0.0      0.0     87.1611    0.0;
    0.0 0.0      0.0      0.0     542.468]

#### 11.3 
Create a `LowerTriangular` matrix from `A` and store it in `A_lowertri`

In [59]:
A_lowertri = LowerTriangular(A)

5×5 LowerTriangular{Int64,Array{Int64,2}}:
 140    ⋅    ⋅    ⋅   ⋅
  97  106    ⋅    ⋅   ⋅
  74   89  152    ⋅   ⋅
 168  131  144   54   ⋅
 131   36   71  142  36

In [ ]:
@assert A_lowertri ==  [140    0    0    0   0;
  97  106    0    0   0;
  74   89  152    0   0;
 168  131  144   54   0;
 131   36   71  142  36]

Please click on `Validate` on the top, once you are done with the exercises.